In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install pandas_profiling 

In [3]:
import pandas_profiling
import nltk
import numpy as np
import cudf as pd
import pandas
import re
import matplotlib.pyplot as plt 
import seaborn as sb
from nltk.corpus import stopwords
import warnings 
warnings.filterwarnings("ignore") #to ignore warnings 
import unidecode #takes Unicode data and tries to represent it in ASCII characters
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
from nltk.stem import PorterStemmer
nltk.download('punkt') #sentence tokenizer 
from nltk.tokenize import word_tokenize 
import matplotlib.animation as animation
import operator #imports efficient functions corresponding to intrinsic operator in python 
import plotly.express as px #high level wrapper for plotly. in other words plotly with simpler syntax 
%matplotlib inline

In [4]:
df=pd.read_csv('../input/twitter-dataset/train_E6oV3lV.csv')

In [5]:
df.head()

In [6]:
df['tweet'][0]

***Removing duplicates and missing values***

In [7]:
df.drop_duplicates(inplace=True)#inplace=True modifies the original dataset instead of returning a new one 

In [8]:
df['label'].isna().sum()

In [9]:
df['tweet'].isna().sum()

In [10]:
df.head(10)

***Removing special characters and linguistic inconsistencies***

In [11]:
#removing @
# if isinstance(pd.DataFrame):
df=df.to_pandas()
for i, tweet in enumerate(df['tweet']):
    lst=tweet.split()
    lst = [i for i in lst if not i.startswith('@')]
    lst=' '.join(lst)
    df['tweet'][i]=lst

In [12]:
df['tweet'][10]

In [13]:
for i, tweet in enumerate(df['tweet']):
    lst=tweet.split()
    lst = [i for i in lst if not i.isdigit()]
    lst=' '.join(lst)
    df['tweet'][i]=lst

In [14]:
df['tweet'][5461]

In [15]:
#Removing the word 'hmm' and it's variants
import re
df['clean_tweet'] = df['tweet'].apply(lambda x : ' '.join([word for word in x.split() if not len(re.findall('h(m)+',word))]))

In [16]:
df['clean_tweet'][5416]

In [17]:
#Code for removing slang words
d = {'luv':'love','wud':'would','lyk':'like','wateva':'whatever','ttyl':'talk to you later',
               'kul':'cool','fyn':'fine','omg':'oh my god!','fam':'family','bruh':'brother',
               'cud':'could','fud':'food'}

In [18]:
df['clean_tweet'] = df['clean_tweet'].apply(lambda x : ' '.join(d[word] if word in d else word for word in x.split()))

In [19]:
df['clean_tweet'][26605]

In [20]:
pip install unidecode 

In [21]:
#Removing all the greek characters using unidecode library
df['clean_tweet'] = df['clean_tweet'].apply(lambda x : ' '.join([unidecode.unidecode(word) for word in x.split()])) 

In [22]:
df['clean_tweet'][7]

In [23]:
#Finding words with '#' 
df['hash']=df['clean_tweet'].apply(lambda x:' '.join([word for word in x.split() if word.startswith('#')]))

In [24]:
df.head()

In [25]:
for i, hashtag in enumerate(df['hash']):
    if hashtag == '':
        df['hash'][i]='no hashtag'

In [26]:
df.head(10)

In [27]:
#remove stopwords
df['clean_tweet'] = df['clean_tweet'].apply(lambda x : ' '.join([word for word in x.split() if not word in set(stopwords.words('english'))]))

In [28]:
dataframe=df.copy(deep=True)

In [29]:
#Lemmitization
lemmatizer = WordNetLemmatizer()
df['clean_tweet'] = df['clean_tweet'].apply(lambda x : ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

In [30]:
df.compare(dataframe)

In [31]:
dataframe['clean_tweet'][0]

In [32]:
df['clean_tweet'][0]

In [33]:
#Stemming
ps = PorterStemmer()
df['clean_tweet'] = df['clean_tweet'].apply(lambda x : ' '.join([ps.stem(word) for word in x.split()]))

In [34]:
df['clean_tweet'][0]

In [35]:
corpus = df['clean_tweet'].to_list()

In [36]:
corpus 

In [37]:
!pip install wordcloud

In [38]:
normal_words = ' '.join([tweet for tweet in df['clean_tweet'][df['label'] == 0]])
wordcloud = WordCloud(width = 800, height = 500, max_font_size = 110,max_words = 100).generate(normal_words)
plt.figure(figsize= (12,8))
plt.imshow(wordcloud, interpolation = 'bilinear',cmap='viridis')
plt.axis('off')

In [39]:
# normalwords=[]
# for i, tweet in enumerate(df['clean_tweet']):
#     if df['label'][i]==0:
#         normalwords.append(tweet)
# ' '.join(normalwords)

In [40]:
normalwords=''.join(tweet for tweet in df['clean_tweet'][df['label']==1])
wordcloud=WordCloud(width = 800, height = 500, max_font_size = 110,max_words = 100).generate(normalwords)
plt.figure(figsize= (12,8))
plt.imshow(wordcloud, interpolation = 'bilinear',cmap='viridis')
plt.axis('off')

In [41]:
#Collecting positive hashtags
hash_positive = []
hash_negative = []

In [42]:
def hashtag_extract(tweets):
    hashtags = []
    # Loop over the words in the tweet
    for tweet in tweets:
        ht = re.findall(r"#(\w+)", tweet) #ht is a list of all the hashtags in tweet 
        hashtags.append(ht)

    return hashtags #hashtags is a list of ht's which too are lists. so a list of lists 

In [43]:
#extracting hashtags from non-hate tweets
hash_positive = hashtag_extract(df['clean_tweet'][df['label'] == 0])

# extracting hashtags from racist/sexist tweets
hash_negative = hashtag_extract(df['clean_tweet'][df['label'] == 1])

In [44]:
# Converting a multidimensional list to a 1-D list
hash_positive = sum(hash_positive,[]) #sum function will turn hash_positive to a 1D list 
hash_negative = sum(hash_negative,[])

In [45]:
from collections import Counter

In [46]:
freq_positive = Counter(hash_positive) #Counter class creates a hashtable of an iterable 
freq_positive = dict(freq_positive.most_common()) #ordered in order of the highest frequencies using most_common()

In [47]:
values_positive=list(freq_positive.values()) #list of frequency value for each hashtag key
values_positive[:5]

In [48]:
key_positive=list(freq_positive.keys())
key_positive[:5]

In [49]:
freq_negative = Counter(hash_negative) #Counter class creates a hashtable of an iterable 
freq_negative = dict(freq_negative.most_common()) #ordered in order of the highest frequencies using most_common()

In [50]:
values_negative=list(freq_negative.values()) #list of frequency value for each hashtag key
values_negative[:5]

In [51]:
key_negative=list(freq_negative.keys())
key_negative[:5]

In [52]:
hashone=pd.DataFrame(key_positive,columns=['Positive_words'])
hashtwo=pd.DataFrame(values_positive,columns=['Positive_count'])
hashthree=pd.DataFrame(key_negative,columns=['Negative_words'])
hashfour=pd.DataFrame(values_negative,columns=['Negative_count'])

In [53]:
hashwords=pd.concat([hashone, hashtwo, hashthree, hashfour], axis=1)

In [54]:
hashwords.head()

In [55]:
df.head()

In [56]:
#Techniques to convert the tweets into Bag-of-Words, TF-IDF
#Building various classifiers: -
#TF-IDF approach
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2,stop_words='english')
# TF-IDF feature matrix
X1 = tfidf_vectorizer.fit_transform(corpus).toarray()
Y1 = df.loc[:,'label'].values

In [57]:
Y1.shape

In [58]:
from sklearn.model_selection import train_test_split
X1_train, X1_test, Y1_train, Y1_test = train_test_split(df['clean_tweet'], df['label'], test_size = 0.3, random_state=0, shuffle = True, stratify=df['label'])
vectorizer = TfidfVectorizer()
X1_train_vect = vectorizer.fit_transform(X1_train)
Y1 = df.loc[:,'label'].values

In [59]:
X1_train

In [60]:
X1_train_vect

In [61]:
#Random Forest using pipelines
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
rf = Pipeline([('tfidf', TfidfVectorizer()), ('rf', RandomForestClassifier())])
rf.fit(X1_train, Y1_train)
y_pred = rf.predict(X1_test)
print(pandas.crosstab(Y1_test,y_pred,rownames=['Actual'],colnames=['Predicted']))
print(classification_report(Y1_test, y_pred))

In [62]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(Y1_test, y_pred))

In [63]:
from sklearn.linear_model import LogisticRegression
clf = Pipeline([('tfidf', TfidfVectorizer()), ('lr', LogisticRegression())])
clf.fit(X1_train, Y1_train)
y_pred = clf.predict(X1_test)
print(pandas.crosstab(Y1_test,y_pred,rownames=['Actual'],colnames=['Predicted']))
print(classification_report(Y1_test, y_pred))

In [64]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(Y1_test, y_pred))

In [66]:
from sklearn.naive_bayes import MultinomialNB
nav = Pipeline([('tfidf', TfidfVectorizer()), ('nv', MultinomialNB())])
nav.fit(X1_train, Y1_train)
y_pred = nav.predict(X1_test)
print(pandas.crosstab(Y1_test,y_pred,rownames=['Actual'],colnames=['Predicted']))
print(classification_report(Y1_test, y_pred))

In [67]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(Y1_test, y_pred))